# Future Strategy - High Frequency Strategy 0001

## 1. Import Library

In [13]:
import dai
import random
import pandas as pd

## 2. Prepare Strategy Data

In [14]:
sd = '2025-10-15'
ed = '2025-10-30'

In [15]:
instrument_list = ['jm8888.DCE']

In [16]:
sql_strategy =  f"""
WITH
data_origin AS (
    SELECT
        date,
        instrument,
        m_ta_ema(close, 5) AS ma_short,
        m_ta_ema(close, 20) AS ma_long,
        IF(ma_short > ma_long AND m_lag(ma_short, 1) < m_lag(ma_long, 1), 1, 0) AS signal_long,
        IF(ma_short < ma_long AND m_lag(ma_short, 1) > m_lag(ma_long, 1), 1, 0) AS signal_short,
    FROM cn_future_bar1m
)
SELECT
    date,
    instrument,
    signal_long,
    signal_short
FROM data_origin
QUALIFY (signal_long = 1 OR signal_short = 1)
ORDER BY date, instrument
"""

In [17]:
df = dai.query(sql_strategy, filters={"date":[sd, ed], "instrument":instrument_list}).df()
df

,date,instrument,signal_long,signal_short
0,2025-10-15 09:36:00,jm8888.DCE,0,1
1,2025-10-15 09:44:00,jm8888.DCE,1,0
2,2025-10-15 10:13:00,jm8888.DCE,0,1
3,2025-10-15 10:15:00,jm8888.DCE,1,0
4,2025-10-15 10:31:00,jm8888.DCE,0,1
...,...,...,...,...
261,2025-10-29 21:47:00,jm8888.DCE,1,0
262,2025-10-29 21:59:00,jm8888.DCE,0,1
263,2025-10-29 22:02:00,jm8888.DCE,1,0
264,2025-10-29 22:45:00,jm8888.DCE,0,1


## 3. Backtest Strategy

In [18]:
from bigmodule import M

def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))

def BigTrader_Before_Trading(context, data):
    context.subscribe_bar(context.instruments, '1m')

def BigTrader_Handle_Tick(context, tick):
    pass

def BigTrader_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d %H:%M:%S")]
    
    from bigtrader.constant import OrderType
    from bigtrader.constant import Direction

    instruments_hold  = set(context.get_account_positions().keys())

    for ins in instruments_hold:
        position_long  = context.get_position(ins, Direction.LONG)
        position_short = context.get_position(ins, Direction.SHORT)
        price = data.current(ins,"open")
        if (position_long.current_qty  != 0):
            context.sell_close(ins, position_long.avail_qty, price, order_type=OrderType.MARKET)
        if (position_short.current_qty != 0):
            context.buy_close(ins, position_short.avail_qty, price, order_type=OrderType.MARKET)

    if len(df_now) == 0:
        return

    instruments_long  = set(df_now[df_now["signal_long"]  == 1]["instrument"])
    instruments_short = set(df_now[df_now["signal_short"] == 1]["instrument"])

    for ins in instruments_long:
        price = data.current(ins, "open")
        volume = 1
        context.buy_open(ins, volume, price, order_type=OrderType.MARKET)
    
    for ins in instruments_short:
        price = data.current(ins, "open")
        volume = 1
        context.sell_open(ins, volume, price, order_type=OrderType.MARKET)

def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 10000 + random.uniform(0, 10),
    frequency="""minute""",
    product_type="""期货""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""open""",
    order_price_field_sell="""open""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

BigTrader.raw_perf.read()

[2026-02-27 11:05:56] [info     ] bigtrader.v34 开始运行 ..
[2026-02-27 11:05:56] [info     ] 2025-10-15, 2025-10-29, , future, instruments=1
[2026-02-27 11:05:56] [info     ] bigtrader module V2.2.0
[2026-02-27 11:05:56] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10
[2026-02-27 11:05:57] [info     ] backtest done, raw_perf_ds:dai.DataSource("_b997fce9718d43e0a2374af424d40c95")


[2026-02-27 11:05:57] [info     ] bigtrader.v34 运行完成 [1.526s].


,date,trading_days,portfolio_value,starting_value,ending_value,starting_cash,ending_cash,returns,algorithm_period_return,benchmark_period_return,...,today_sell_balance,commission,capital_changed,orders,transactions,positions,POS_FAC,TRA_FAC,LOG,benchmark_returns
2025-10-15,2025-10-15,1,9536.585555,0.0,0.0,10008.585555,9536.585555,-0.047160,-0.047160,0.000000,...,14891.5,52.0,0.0,"[{'symbol': 'jm8888.DCE', 'id': '1', 'dt': 202...","[{'symbol': 'jm8888.DCE', 'dt': 2025-10-15 09:...","[{'symbol': 'jm8888.DCE', 'direction': '1', 'c...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[09:36:00][id:...",0.000000
2025-10-16,2025-10-16,2,9102.585555,0.0,0.0,9536.585555,9102.585555,-0.045509,-0.090522,0.002634,...,30092.0,104.0,0.0,"[{'symbol': 'jm8888.DCE', 'id': '27', 'dt': 20...","[{'symbol': 'jm8888.DCE', 'dt': 2025-10-15 21:...","[{'symbol': 'jm8888.DCE', 'direction': '1', 'c...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[21:03:00][id:...",0.002634
2025-10-17,2025-10-17,3,8342.585555,0.0,0.0,9102.585555,8342.585555,-0.083493,-0.166457,-0.019984,...,29607.0,100.0,0.0,"[{'symbol': 'jm8888.DCE', 'id': '79', 'dt': 20...","[{'symbol': 'jm8888.DCE', 'dt': 2025-10-16 21:...","[{'symbol': 'jm8888.DCE', 'direction': '1', 'c...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[21:42:00][id:...",-0.022559
2025-10-20,2025-10-20,4,8562.585555,0.0,0.0,8342.585555,8562.585555,0.026371,-0.144476,-0.014777,...,24459.5,80.0,0.0,"[{'symbol': 'jm8888.DCE', 'id': '129', 'dt': 2...","[{'symbol': 'jm8888.DCE', 'dt': 2025-10-17 21:...","[{'symbol': 'jm8888.DCE', 'direction': '1', 'c...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[21:01:00][id:...",0.005313
2025-10-21,2025-10-21,5,8586.585555,0.0,0.0,8562.585555,8586.585555,0.002803,-0.142078,0.000344,...,28701.0,96.0,0.0,"[{'symbol': 'jm8888.DCE', 'id': '170', 'dt': 2...","[{'symbol': 'jm8888.DCE', 'dt': 2025-10-20 21:...","[{'symbol': 'jm8888.DCE', 'direction': '1', 'c...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[21:01:00][id:...",0.015348
2025-10-22,2025-10-22,6,8140.585555,0.0,0.0,8586.585555,8140.585555,-0.051941,-0.186640,-0.002978,...,34692.0,116.0,0.0,"[{'symbol': 'jm8888.DCE', 'id': '218', 'dt': 2...","[{'symbol': 'jm8888.DCE', 'dt': 2025-10-21 21:...","[{'symbol': 'jm8888.DCE', 'direction': '1', 'c...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[21:09:00][id:...",-0.003321
2025-10-23,2025-10-23,7,7832.585555,0.0,0.0,8140.585555,7832.585555,-0.037835,-0.217413,0.000012,...,20786.5,68.0,0.0,"[{'symbol': 'jm8888.DCE', 'id': '276', 'dt': 2...","[{'symbol': 'jm8888.DCE', 'dt': 2025-10-22 21:...","[{'symbol': 'jm8888.DCE', 'direction': '1', 'c...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[21:16:00][id:...",0.002999
2025-10-24,2025-10-24,8,7636.585555,0.0,0.0,7832.585555,7636.585555,-0.025024,-0.236997,0.011809,...,23775.0,76.0,0.0,"[{'symbol': 'jm8888.DCE', 'id': '310', 'dt': 2...","[{'symbol': 'jm8888.DCE', 'dt': 2025-10-23 21:...","[{'symbol': 'jm8888.DCE', 'direction': '1', 'c...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[21:12:00][id:...",0.011797
2025-10-27,2025-10-27,9,7244.585555,0.0,0.0,7636.585555,7244.585555,-0.051332,-0.276163,0.023822,...,28781.0,92.0,0.0,"[{'symbol': 'jm8888.DCE', 'id': '348', 'dt': 2...","[{'symbol': 'jm8888.DCE', 'dt': 2025-10-24 21:...","[{'symbol': 'jm8888.DCE', 'direction': '1', 'c...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[21:10:00][id:...",0.011873
2025-10-28,2025-10-28,10,7102.585555,0.0,0.0,7244.585555,7102.585555,-0.019601,-0.290351,0.018602,...,35257.5,112.0,0.0,"[{'symbol': 'jm8888.DCE', 'id': '394', 'dt': 2...","[{'symbol': 'jm8888.DCE', 'dt': 2025-10-27 21:...","[{'symbol': 'jm8888.DCE', 'direction': '1', 'c...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[21:16:00][id:...",-0.005099
